# Database Seeder
This notebook seeds the `Japlayer` SQLite database with a dozen of example entries for testing purposes.

### Import dependencies and namespaces

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "..\Japlayer.Data\bin\Debug\net8.0\Japlayer.Data.dll"
#r "..\Japlayer.WinUI\bin\Debug\net8.0-windows10.0.19041.0\Japlayer.dll"

using Japlayer.Data.Context;
using Japlayer.Data.Entities;
using Japlayer.Models;
using Microsoft.EntityFrameworkCore;
using System.IO;
using System.Text.Json;
using System.Collections.Generic;
using System.Linq;

### Initialize Database Context

In [ ]:
var appDataPath = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.LocalApplicationData), "Japlayer");
var dbPath = Path.Combine(appDataPath, "medias.db");

var optionsBuilder = new DbContextOptionsBuilder<DatabaseContext>();
optionsBuilder.UseSqlite($"Data Source={dbPath}");

var context = new DatabaseContext(optionsBuilder.Options);
await context.Database.EnsureCreatedAsync();
Console.WriteLine($"✓ Initialized context for: {dbPath}");

### Clear Existing Data (Optional)
Use the cell below if you want to start with a clean slate. Uses PRAGMA to avoid Foreign Key constraint errors.

In [ ]:
// Disable foreign keys temporarily to avoid deletion order issues
context.Database.EnsureDeleted();
context.Database.EnsureCreated();

Console.WriteLine("✓ Database cleared.");

### Seed Lookup Tables
Creates Studios, Series, Genres, People, and Tags. 

In [ ]:
var initStudios = new List<MediaStudio> {
    new() { Name = "Digital Frontier" },
    new() { Name = "Crystal Vision Studios" },
    new() { Name = "Neon Horizon Prod" },
    new() { Name = "Midnight Cinema" },
    new() { Name = "Infinity Motion" }
};

var initSeries = new List<MediaSeries> {
    new() { Name = "Chronicles of Aether" },
    new() { Name = "Cyber City Chronicles" },
    new() { Name = "Magical Nights" },
    new() { Name = "Starbound Voyagers" }
};

var initGenres = new List<MediaGenre> {
    new() { Name = "Action" },
    new() { Name = "Adventure" },
    new() { Name = "Comedy" },
    new() { Name = "Drama" },
    new() { Name = "Fantasy" },
    new() { Name = "Romance" },
    new() { Name = "Sci-Fi" },
    new() { Name = "Thriller" }
};

var initPeople = new List<MediaPerson> {
    new() { Name = "Alice Johnson" },
    new() { Name = "Bob Smith" },
    new() { Name = "Charlie Brown" },
    new() { Name = "Diana Prince" },
    new() { Name = "Edward Elric" },
    new() { Name = "Fiona Gallagher" },
    new() { Name = "George Costanza" },
    new() { Name = "Hannah Montana" }
};

var initTags = new List<UserTag> {
    new() { Name = "Must Watch" },
    new() { Name = "Favorite" },
    new() { Name = "Watch Later" },
    new() { Name = "Finished" }
};

context.MediaStudios.AddRange(initStudios);
context.MediaSeries.AddRange(initSeries);
context.MediaGenres.AddRange(initGenres);
context.MediaPeople.AddRange(initPeople);
context.UserTags.AddRange(initTags);

await context.SaveChangesAsync();
Console.WriteLine("✓ Lookup tables seeded.");

### Seed Media Entries
Creates 12 media entries with associated metadata, locations, and images. Re-fetches lookup data to ensure tracking.

In [ ]:
// Re-fetch lookups to ensure they are tracked by the current context instance
var studios = await context.MediaStudios.ToListAsync();
var series = await context.MediaSeries.ToListAsync();
var genres = await context.MediaGenres.ToListAsync();
var people = await context.MediaPeople.ToListAsync();
var tags = await context.UserTags.ToListAsync();

var random = new Random();

for (int i = 1; i <= 12; i++)
{
    var mediaId = $"M-{i:D3}";
    var media = new Media { MediaId = mediaId };
    
    // Add the media to context immediately so related collections find its MediaId correctly
    context.Media.Add(media);

    // Assign random properties (unique within this item)
    foreach (var g in genres.OrderBy(x => random.Next()).Take(2)) media.Genres.Add(g);
    media.Studios.Add(studios[random.Next(studios.Count)]);
    media.Series.Add(series[random.Next(series.Count)]);
    
    // Assign Cast
    foreach (var p in people.OrderBy(x => random.Next()).Take(2)) media.People.Add(p);
    
    // Assign Staff
    media.PeopleNavigation.Add(people[random.Next(people.Count)]);
    
    // Locations
    media.MediaLocations.Add(new MediaLocation 
    {
        Scene = 1, 
        Hostname = "Localhost", 
        Path = @"C:\Media\Example_" + i + "_Part1.mp4"
    });
    
    if (random.Next(2) == 0)
    {
        media.MediaLocations.Add(new MediaLocation 
        {
            Scene = 2, 
            Hostname = "Localhost", 
            Path = @"C:\Media\Example_" + i + "_Part2.mp4"
        });
    }

    // Images
    var coverImage = new MediaImage 
    {
        Filepath = $"C:\\Images\\{mediaId}_cover.jpg", 
        Url = $"https://picsum.photos/seed/{mediaId}_cover/400/600"
    };
    var thumbImage = new MediaImage 
    {
        Filepath = $"C:\\Images\\{mediaId}_thumb.jpg", 
        Url = $"https://picsum.photos/seed/{mediaId}_thumb/200/300"
    };
    
    media.MediaImages.Add(coverImage);
    media.MediaImages.Add(thumbImage);

    // Metadata
    media.MediaMetadata.Add(new MediaMetadata
    {
        MetadataUrl = $"https://example.com/media/{mediaId}",
        ContentId = $"CID-{i:D5}",
        Title = $"Example Media Title {i}",
        ReleaseDate = new DateOnly(2020 + random.Next(6), random.Next(1, 13), random.Next(1, 28)),
        RuntimeMinutes = 20 + random.Next(100),
        CoverNavigation = coverImage,
        ThumbnailNavigation = thumbImage
    });
}

await context.SaveChangesAsync();
Console.WriteLine("✓ 12 Media entries seeded.");

### Final Check
Display the total count of entries per table.

In [ ]:
var counts = new []
{
    new { Table = "Media", Count = await context.Media.CountAsync() },
    new { Table = "Metadata", Count = await context.MediaMetadata.CountAsync() },
    new { Table = "Locations", Count = await context.MediaLocations.CountAsync() },
    new { Table = "Images", Count = await context.MediaImages.CountAsync() },
    new { Table = "Genres", Count = await context.MediaGenres.CountAsync() },
    new { Table = "People", Count = await context.MediaPeople.CountAsync() },
    new { Table = "Series", Count = await context.MediaSeries.CountAsync() },
    new { Table = "Studios", Count = await context.MediaStudios.CountAsync() },
    new { Table = "UserTags", Count = await context.UserTags.CountAsync() }
};

counts